ADP data by Steven Cunningham 06/16/17

This code is intened for take SonTek ADP data
- First determine seafloor by signal amplitude file ".a#". The signal should slowly decay through water, when decay is
  interupted with amplitude spike, then the signal is bouncing back from the floor. 
- Compare one bin to the previous until the next bin is larger than previous.
- Number of good profiles will be used on velocity data. 

In [1]:
import pandas as pd
import numpy as np
import glob
import matplotlib.pyplot as plt
from scipy import stats
import scipy
import csv


In [67]:
sig_amp = 'SWC0610130945.a2'
df = pd.read_csv(sig_amp, delimiter='\s+', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,1,147,134,121,121,121,140,180,144,92,...,32,40,52,48,43,33,28,27,26,26
1,2,148,132,118,115,117,141,185,148,95,...,31,39,54,48,41,32,27,25,24,24
2,3,149,145,125,119,114,142,178,157,101,...,31,35,50,49,42,32,26,25,24,24
3,4,147,137,131,117,118,129,178,156,84,...,31,33,47,56,41,29,25,24,24,24
4,5,151,139,137,108,114,124,176,155,75,...,31,31,45,57,40,28,25,24,24,24


In [70]:
mean = np.mean(df, axis = 0)[1:] # average profiles across a depth bin
mean2 = pd.Series.to_frame(mean, 'average_profile') # transform to dataframe 
mean2["next_profile"] = mean2["average_profile"].shift(-1) # Shift down to commpare difference 
mean2["Nartural_decay"] = mean2["average_profile"] > mean2["next_profile"] # check if the next bin is greater than previous 
mean2["True_profile"] = mean2["Nartural_decay"].cumsum() # gives values to groups of true/false
mean2["Number"] = np.arange(23)[1:] # make a range of values 
mean2["good_profile"] = mean2["True_profile"] == mean2["Number"] # see if values assigned to true/false match number order  
print(mean2)

how_many=sum(mean2["good_profile"]) # how many good profiles are there. This will be used on velocity data 
mean3=mean2["average_profile"][:how_many+1] # get profile values for good bins. Add 1 because the .shift 


    average_profile  next_profile Nartural_decay  True_profile  Number  \
1        149.285714    139.642857           True             1       1   
2        139.642857    128.785714           True             2       2   
3        128.785714    119.428571           True             3       3   
4        119.428571    117.642857           True             4       4   
5        117.642857    135.571429          False             4       5   
6        135.571429    176.428571          False             4       6   
7        176.428571    149.428571           True             5       7   
8        149.428571     88.500000           True             6       8   
9         88.500000     57.071429           True             7       9   
10        57.071429     42.285714           True             8      10   
11        42.285714     34.857143           True             9      11   
12        34.857143     31.714286           True            10      12   
13        31.714286     35.857143     

4
